# Train model to detect images with an obstacle on track and make a lite version of the model

Author : Johan Jublanc
    
Date : 22/09/2019

Description : 

Use the Python interpreter to load a .tflite file and run inference

_TODO :_
* Generalize this procedure to each model in the car

## Imports

In [ ]:
import numpy as np
import tensorflow as tf

from xebikart.images import transformer as T

import xebikart.dataset as dataset

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.compat.v1 import lite

In [ ]:
tf.enable_eager_execution()

## Params

In [ ]:
model_name = "detection_model"
resize_shape = (160,160)

# Load and convert the keras model (.h5)

The model keras (.h5) is loaded and convert to a lite model (.tflite) which is lighter :
* a smaller file thank to the serialization library FlatBuffer
* a model smaller by reducing the precision of the numbers in the model

See : 
* https://www.tensorflow.org/lite/performance/post_training_quantization
* https://www.tensorflow.org/lite/convert/index

First create a converter using the keras model as input

In [ ]:
try:
    converter = lite.TFLiteConverter.from_keras_model_file(model_name + ".h5")
except:
    print("No model named {}.h5 in the current folder".format(model_name))

Then we parametrize an optimizer to quantize the model so that it is smaller

In [ ]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

Then convert the model

In [ ]:
tflite_model = converter.convert()

And finally save the lite version of the model

In [ ]:
open("{}.tflite".format(model_name), "wb").write(tflite_model)

# Define the inference process

With tflite models, inferences are made thanks to an __interpreter__ that is fast and lean. We will define some functions to handle the whole prediction process with the Python API : 
* first, prepare the image -> read/normalize/resize/reshape/convert
* second, define an interpreter and input and ouput details
* finally, define a predictor 

In [ ]:
def prepare_image(image_path, resize_shape=(160,160)):
    
    tf_image = T.read_image(image_path)
    tf_image = T.normalize(tf_image)
    tf_image = tf.image.resize(tf_image,resize_shape, method = 2)
    tf_image = tf.reshape(tf_image,(1,resize_shape[0],resize_shape[1],3))
    tf_image = np.array(tf_image)
    
    return tf_image

In [ ]:
def interpreter_and_details(model_path) :

    # Load TFLite model and allocate tensors

    interpreter = tf.lite.Interpreter(model_path = model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_shape = input_details[0]['shape']
    
    return interpreter, input_details, output_details, input_shape

In [ ]:
interpreter, input_details, output_details, input_shape = interpreter_and_details(model_name + ".tflite")

In [ ]:
def predictor_builder(interpreter, input_details, output_details, input_shape):
    def predictor(input_image):
        interpreter.set_tensor(input_details[0]['index'], input_image)
        interpreter.invoke()

        # The function `get_tensor()` returns a copy of the tensor data.
        # Use `tensor()` in order to get a pointer to the tensor.
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        return output_data[0][0]
    return predictor

In [ ]:
predictor = predictor_builder(interpreter, input_details, output_details, input_shape)

## Test the inference process on a random image

Get the data

In [ ]:
# dataset parameters
tubes_root_folder = "file:/workspace/xebikart-ml-tubes"
tubes_folders = [
    "tub.v4.02",
    "tub.v6.01"
]

In [ ]:
raw_tubes_df = dataset.get_tubes_df(tubes_root_folder, tubes_folders, tubes_extension=".tar.gz")
tubes_df = raw_tubes_df.rename(columns={"cam/image_array": "images_path", "user/angle": "angles", "user/throttle": "throttles"})

Randomly select 4 images

In [ ]:
random_image_path = []
for i in range(4):
    random_image_path.append(tubes_df.sample()["images_path"].values[0])

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15,15), constrained_layout=True)

for i in range(4):
    # function pre-defined are used to compute the prediction
    image = prepare_image(random_image_path[i])
    prediction = predictor(image)
    
    # eaxh image in shown with the prediction
    axs[i].set_title("Prediction = {}".format(prediction))
    axs[i].imshow(image.reshape(resize_shape[0],resize_shape[1],3))